<a href="https://colab.research.google.com/github/adalbertii/azure-cognitive-service/blob/main/form_recognizer_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install azure-ai-formrecognizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [3]:


from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

import os


In [11]:
# Endpoint URL
endpoint = r"https://form-reco-01.cognitiveservices.azure.com/"
apim_key = "2492c8adf163495da0aeb667187d85bd"
post_url = endpoint + "/formrecognizer/v2.1/prebuilt/receipt/analyze"
source = r"/content/drive/MyDrive/dane/Receipt01.jpg"

headers = {
# Request headers
  'Content-Type': '<file type>',
  'Ocp-Apim-Subscription-Key': apim_key,
}

params = {
  "includeTextDetails":True,
  "locale": "en-US"
}





In [19]:
    ########### Python Form Recognizer Async Receipt #############

import json
import time
from requests import get, post

    # Endpoint URL
endpoint = r"https://form-reco-01.cognitiveservices.azure.com/"
apim_key = "2492c8adf163495da0aeb667187d85bd"
post_url = endpoint + "/formrecognizer/v2.1/prebuilt/receipt/analyze"
source = "/content/drive/MyDrive/dane/contoso-allinone.jpg"

headers = {
    # Request headers
    'Content-Type': 'application/octet-stream',
    'Ocp-Apim-Subscription-Key': apim_key,
}

params = {
    "includeTextDetails": True,
    "locale": "en-US"
}

f= open(source, "rb")
data_bytes = f.read()



In [20]:
resp = post(url = post_url, data = data_bytes, headers = headers, params = params)
if resp.status_code != 202:
  print("POST analyze failed:\n%s" % resp.text)
  quit()
else:
  print("POST analyze succeeded:\n%s" % resp.headers)
 # get_url = resp.headers["operation-location"]


POST analyze succeeded:
{'Content-Length': '0', 'Operation-Location': 'https://form-reco-01.cognitiveservices.azure.com/formrecognizer/v2.1/prebuilt/receipt/analyzeResults/9ad6b224-6d17-4fc3-a8d3-d7f2a788b7ef', 'x-envoy-upstream-service-time': '129', 'apim-request-id': '9ad6b224-6d17-4fc3-a8d3-d7f2a788b7ef', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'East US', 'Date': 'Mon, 20 Nov 2023 09:25:42 GMT'}


In [30]:
wynik =  resp.raw

wynik